In [17]:
from fastapi import FastAPI, UploadFile, File
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import asyncio


In [18]:
app = FastAPI()

MODEL = tf.keras.models.load_model("../saved_model/1/cofeedisease.h5")

CLASS_NAMES = ["phoma", "leaf rust", "Cerscospora"]

C:\Users\ijas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\framework\ops.py:1936: RuntimeWarning: coroutine 'Server.serve' was never awaited
  with graph._c_graph.get() as c_graph:


In [19]:
@app.get("/ping")
async def ping():
    return {"HelloWorld"}

In [20]:
def read_file_as_image(data):
    image = np.array(Image.open(BytesIO(data)))
    return image


In [21]:
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    image = read_file_as_image(await file.read())
    image_batch = np.expand_dims(image, 0)
    predictions = MODEL.predict(image_batch)
    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = float(np.max(predictions[0]))
    return {
        'class': predicted_class,
        'confidence': float(confidence)
    }

In [22]:
async def start_server():
    app = FastAPI()
    # Define routes and load model here
    uvicorn_options = {"host": "localhost", "port": 8000}
    await uvicorn.run(app, **uvicorn_options)


In [25]:
if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    asyncio.run_coroutine_threadsafe(start_server(), loop)